In [24]:
import numpy as np
from scipy.stats import norm

class MonteCarloOptionPricing:
    def __init__(self, S0, K, T, r, sigma, n_simulations, n_steps, option_type="call", asian=True):
        """
        Initialize the Monte Carlo pricing model.
        
        Parameters:
        S0 : float  - Initial stock price
        K : float   - Strike price
        T : float   - Time to maturity (years)
        r : float   - Risk-free interest rate (annualized)
        sigma : float - Volatility of underlying asset (annualized)
        n_simulations : int - Number of Monte Carlo simulations
        n_steps : int - Number of time steps in each simulation
        option_type : str - "call" or "put"
        asian : bool - If True, prices an Asian option; otherwise, a European option.
        """
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.n_simulations = n_simulations
        self.n_steps = n_steps
        self.dt = T / n_steps
        self.option_type = option_type
        self.asian = asian
    
    def simulate_asset_paths(self):
        """Simulates asset price paths using Geometric Brownian Motion (GBM)."""
        S = np.zeros((self.n_simulations, self.n_steps + 1))
        S[:, 0] = self.S0  # Initial stock price
        
        # Generate random numbers for Brownian motion
        Z = np.random.randn(self.n_simulations, self.n_steps)
        
        for t in range(1, self.n_steps + 1):
            S[:, t] = S[:, t-1] * np.exp((self.r - 0.5 * self.sigma**2) * self.dt + 
                                         self.sigma * np.sqrt(self.dt) * Z[:, t-1])
        return S
    
    def calculate_payoff(self, S):
        """Calculates option payoff based on Asian or European option type."""
        if self.asian:
            avg_price = np.mean(S[:, 1:], axis=1)  # Exclude initial price
        else:
            avg_price = S[:, -1]  # Final price for European option
        
        if self.option_type == "call":
            payoff = np.maximum(avg_price - self.K, 0)
        else:
            payoff = np.maximum(self.K - avg_price, 0)
        
        return payoff
    
    def price_option(self):
        """Prices the option using Monte Carlo simulation."""
        S = self.simulate_asset_paths()
        payoff = self.calculate_payoff(S)
        
        # Discount expected payoff
        option_price = np.exp(-self.r * self.T) * np.mean(payoff)
        std_error = np.std(payoff) / np.sqrt(self.n_simulations)
        
        return option_price, std_error


In [25]:
# Set parameters
S0 = 100
K = 100
T = 1
r = 0.05
sigma = 0.2
n_simulations = 10000
n_steps = 252
option_type = "call"
asian = True

# Price the option
mc_pricing = MonteCarloOptionPricing(S0, K, T, r, sigma, n_simulations, n_steps, option_type, asian)
price, std_error = mc_pricing.price_option()

# Print the results
print(f"Monte Carlo Estimated Asian Option Price: {price:.4f}")
print(f"Standard Error: {std_error:.4f}")


Monte Carlo Estimated Asian Option Price: 5.7636
Standard Error: 0.0841
